# Keras Network to Classify Odd/Even Numbers

In [1]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

Using TensorFlow backend.


## Big Picture
- Supervised learning, so we have labeled data
- Neural networks, like other supervised algorithms, try to approximate any arbitrary function, given enough data

<img src="https://camo.githubusercontent.com/fedd5d66bea57a430635498de58dc7c6f064f280/68747470733a2f2f64707a6268796262327064636a2e636c6f756466726f6e742e6e65742f63686f6c6c65742f466967757265732f303166696730322e6a7067">

## Goal
- Train a neural network classifier to classify if an integer is odd or even.

## Workflow
1. Setup training data
2. Create binary encoding for the input layer
3. Create binary decoding for the ouput layer
4. Create the model, add layers, and train/fit
5. Create predictions on new data
6. Evaluate the performance of the classifier on train data
7. Evaluate the performance of the classifier on test data

In [2]:
# Specify the number of binary digits. 
NUM_DIGITS = 10

In [3]:
## Setup the training data for 101-1023. 1023 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [4]:
raw_training_data[:10]

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110])

In [5]:
## Binary Encoding function
# In order to binary encode the input values
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [6]:
print("number \t [binary digits]")
print(1, "\t", binary_encode(1, NUM_DIGITS))
print(2, "\t", binary_encode(2, NUM_DIGITS))
print(3, "\t", binary_encode(3, NUM_DIGITS))
print(4, "\t", binary_encode(4, NUM_DIGITS))
print(5, "\t", binary_encode(5, NUM_DIGITS))

number 	 [binary digits]
1 	 [1 0 0 0 0 0 0 0 0 0]
2 	 [0 1 0 0 0 0 0 0 0 0]
3 	 [1 1 0 0 0 0 0 0 0 0]
4 	 [0 0 1 0 0 0 0 0 0 0]
5 	 [1 0 1 0 0 0 0 0 0 0]


In [7]:
# encode the training data
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]

In [8]:
x_train = np.array(encoded_training_data)

In [9]:
x_train

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [10]:
# Encode "even" and "odd" to zero or 1
# If we had more than 2 options, we'd one hot encode to make an array
# One hot encoding the number
# Creating the labels, the y values
def encode_even_odd(i):
    if i % 2 == 0:
        return [0]
    else:
        return [1]

assert encode_even_odd(2) == [0]
assert encode_even_odd(3) == [1]

In [11]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([encode_even_odd(i) for i in range(101, 2 ** NUM_DIGITS)])

In [12]:
print("Number representeed as a decimal", raw_training_data[0])
print("Number represented as binary in a list:", x_train[0])
print("0 for even, 1 for odd:", y_train[0])

Number representeed as a decimal 101
Number represented as binary in a list: [1 0 1 0 0 1 1 0 0 0]
0 for even, 1 for odd: [1]


In [13]:
print("Number representeed as a decimal", raw_training_data[1])
print("Number represented as binary in a list:", x_train[1])
print("0 for even, 1 for odd:", y_train[1])

Number representeed as a decimal 102
Number represented as binary in a list: [0 1 1 0 0 1 1 0 0 0]
0 for even, 1 for odd: [0]


## Compare raw inputs to the lables
- The first five values in the input tensor are 101, 102, 103, 104, and 105
- The encoded labels specify "odd", "even", "odd", "even", and then "odd"
- Remember that training a classifier is supervised learning with labeled data

In [14]:
print("Raw training data", raw_training_data[:5])
print(y_train[:5].flatten())

Raw training data [101 102 103 104 105]
[1 0 1 0 1]


In [15]:
# Convert from binary back to the ouput string
# Decodes the output layer to english
def decode_even_odd(n):
    if n == 0:
        return "even"
    else:
        return "odd"

assert decode_even_odd(0) == 'even'
assert decode_even_odd(1) == 'odd'

In [16]:
# Another way I've seen this decoding written (especially with one hot encoded options)
# Convert from binary back to the ouput string
def decode_even_odd2(n):
    return ["even", "odd"][n]

assert decode_even_odd2(0) == 'even'
assert decode_even_odd2(1) == 'odd'

In [17]:
# Now let's build our model, add layers, compile, and fit it!
# Seauential is a good model for sequence data like 1, 2, 3 dimentional arrays
model = Sequential()

## Add the model's layers.
# These are defaults and they work pretty well
# The only layer we need an input is the first layer
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu")) # relu max(0, x)
model.add(Dense(1000, activation="relu"))

# Almost all of the time, the last layer's first argument is the number of outcomes
model.add(Dense(2, activation="softmax"))

# categorical_crossentropy for multiclass
# should be using a binary classifier
model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])

In [18]:
# Fit the model on the training data
# nb_epoch is the number of training loops is the number of epochs
model.fit(x_train, y_train, nb_epoch=3, batch_size=128)

Epoch 1/3
923/923 [==============================] - 0s 136us/step - loss: 1.0807 - accuracy: 0.7151
Epoch 2/3
923/923 [==============================] - 0s 84us/step - loss: 0.0087 - accuracy: 1.0000
Epoch 3/3
923/923 [==============================] - 0s 71us/step - loss: 0.0014 - accuracy: 1.0000


In [19]:
# Setup x_test for numbers from 1 to 100 (the model hasn't seen these at all)
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))

# model.predict_classes(X_test) or model.predict_classes(out_of_sample_data)
y_test = model.predict_classes(x_test)

In [20]:
# Setup predicted output
predictions = np.vectorize(decode_even_odd)(y_test)
print (predictions)

['odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even']


In [21]:
actual = np.array([])
for i in numbers:
    if i % 2 == 0:
        actual = np.append(actual, "even")
    else:
        actual = np.append(actual, "odd")

actual

array(['odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even'], dtype='<U32')

In [22]:
# Let's evaluate the model's predictions
evaluate = np.array(actual == predictions)
print (np.count_nonzero(evaluate == True) / 100)

1.0


### Ok, 100% accuracy... but what if we send the model random numbers?

In [23]:
random_numbers = np.random.randint(1023, size=10000)
random_numbers[:10]

array([885, 550, 459, 991, 633, 460, 682, 738, 413,  57])

In [24]:
x_test = np.transpose(binary_encode(random_numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [25]:
predictions = np.vectorize(decode_even_odd)(y_test)

In [26]:
predictions.shape

(10000,)

In [27]:
actual = np.array([])
for i in random_numbers:
    if i % 2 == 0:
        actual = np.append(actual, "even")
    else:
        actual = np.append(actual, "odd")

actual

array(['odd', 'even', 'odd', ..., 'even', 'odd', 'even'], dtype='<U32')

In [28]:
actual.shape

(10000,)

In [29]:
# Let's evaluate the model's predictions
evaluate = np.array(actual == predictions)
print (np.count_nonzero(evaluate == True) / 100)

100.0


In [30]:
# make predictions on 1-100 

In [31]:
random_numbers = np.random.randint(101, size=10000)
random_numbers[:20]

array([87, 17, 75, 26, 49, 61, 39, 15, 30, 55, 85, 36, 30, 39, 62,  2, 69,
       77, 63, 52])

In [32]:
x_test = np.transpose(binary_encode(random_numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [33]:
predictions = np.vectorize(decode_even_odd)(y_test)

In [34]:
predictions.shape

(10000,)

In [35]:
actual = np.array([])
for i in random_numbers:
    if i % 2 == 0:
        actual = np.append(actual, 'even')
    else:
        actual = np.append(actual, 'odd')
actual

array(['odd', 'odd', 'odd', ..., 'odd', 'even', 'odd'], dtype='<U32')

In [36]:
actual.shape

(10000,)

In [37]:
evaluate = np.array(actual == predictions)
print(np.count_nonzero(evaluate == True) / 100)

100.0


In [38]:
# make predictions on 1024-2048

In [39]:
random_numbers = np.random.randint(1024, 2049, size=10000)
random_numbers[:20]

array([1540, 1276, 1026, 1259, 1548, 1430, 1061, 1253, 1776, 1066, 1405,
       1388, 1863, 1259, 1110, 1623, 1902, 1592, 1117, 2006])

In [40]:
x_test = np.transpose(binary_encode(random_numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [41]:
predictions = np.vectorize(decode_even_odd)(y_test)
predictions.shape

(10000,)

In [42]:
actual = np.array([])
for i in random_numbers:
    if i % 2 == 0:
        actual = np.append(actual, 'even')
    else:
        actual = np.append(actual, 'odd')
actual

array(['even', 'even', 'even', ..., 'odd', 'odd', 'odd'], dtype='<U32')

In [43]:
actual.shape

(10000,)

In [44]:
evaluate = np.array(actual == predictions)
print(np.count_nonzero(evaluate == True) / 100)

100.0


In [45]:
# make predictions on 2048-4096

In [46]:
random_numbers = np.random.randint(2048, 4097, size=10000)
random_numbers[:20]

array([3622, 3096, 2986, 2741, 3262, 2429, 3578, 2132, 2731, 3203, 2834,
       3872, 3113, 3248, 3127, 3927, 2510, 3711, 2136, 3473])

In [47]:
x_test = np.transpose(binary_encode(random_numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [48]:
predictions = np.vectorize(decode_even_odd)(y_test)
predictions.shape

(10000,)

In [49]:
actual = np.array([])
for i in random_numbers:
    if i % 2 == 0:
        actual = np.append(actual, 'even')
    else:
        actual = np.append(actual, 'odd')
actual

array(['even', 'even', 'even', ..., 'even', 'even', 'odd'], dtype='<U32')

In [50]:
actual.shape

(10000,)

In [51]:
evaluate = np.array(actual == predictions)
print(np.count_nonzero(evaluate == True) / 100)

100.0
